<a href="https://www.kaggle.com/code/emdogan/pii-data-detection-infer-with-w-b-tercume?scriptVersionId=166832873" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# W&B inference code for PII Data Detection competition

As presented in the [live training session](https://www.youtube.com/watch?v=w4ZDwiSXMK0).

The model being inferred here is the one we trained live during the session above!

Training code: https://www.kaggle.com/code/thedrcat/pii-data-detection-train-with-w-b

<div style="background-color:purple; color:white; padding:10px; border-radius: 5px; text-align:center;">
  <h1>Kod blok no: 1</h1>
</div>

In [ ]:
model_path = '/kaggle/input/pii008'
inference_max_length = 768
max_length = 400
doc_stride = 300
threshold=0.9

"""
---------------------------------
- modelin yolu belirlenmiş ve gerekli değerler atanmış

+ model_path değişkenine kullanılacak olan modelin yolu atanmış. 
+ Tahmin yapılırkenki girdi uzunluğu inference_max_length değişkenine 768 olarak girilmiş
    + (768 genellikle BERT gibi modellerin girdi uzunluğuna karşılık gelir)
+ Çıktı uzunluğuna da max_length değişkeni üzerinden 400 olarak atama yapılmış.
+ büyük metinlerde çalışılırken belgeleri küçük metinlere parçalamak gerekebilir, doc_stride
    değişkeni ile bu belgelerin parçalara bölme değeri 300 olarak seçilmiş.
+ Tahminlerin ne kadar güvenilir olması gerektiğini belirlemek için kabul edilebilir 
  güvenilirlik seviyesi belirlenmiş ve burada 0.9 olarak ayarlanmış, yani tahminlerin 
  %90 veya daha fazlasının güvenilir olması gerektiği belirtilmiş.
---------------------------------
"""

In [ ]:
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification, AutoTokenizer

model = AutoModelForTokenClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)

"""
---------------------------------
- model tokenizer ve collator değişkenlerine model üzerinden atama yapılmış

+ Önceden eğitilmiş bir model, model değişkenine atanıyor.
+ tokenizer değişkenine önceden eğitilmiş bir modelden tokenizer ataması yapılıyor.
+ Eğitim sırasında modelin kullancağı veriye uygun bir şekilde veriyi hazırlamak için
  veri koleksiyon nesnesi oluşturuluyor ve belirtilen tokenizer'ı gerekirse belirli 
  bir diziye doldurmak için kullanılan sayıyı 16 olarak alıyor.
---------------------------------
"""

In [ ]:
import json
import pandas as pd

train = json.load(open("../input/pii-detection-removal-from-educational-data/test.json"))
df = pd.DataFrame(train)

def add_token_indices(doc_tokens):
    token_indices = list(range(len(doc_tokens)))
    return token_indices

df['token_indices'] = df['tokens'].apply(add_token_indices)

"""
---------------------------------
- dataset yüklenmiş ve tokenlerin indisi oluşturulup eksütun olarak eklenmiş

+ df değişkenine test dosyası dataframe olarak atanıyor.
+ add_token_indices fonksiyonu belge için token dizinlerini yani indekslerini oluşturuyor ve 
   token_indices değişkenine atıyor.
+ dataframe'deki her token, fonksiyonun uygulanması ile indeks numarası alıyor ve
   dataframe'deki 'token_indices' sütununa bu indeks numaraları ile atanıyor.
---------------------------------
"""

# token indices eklenmiş df çıktısı:
# df.head(10)

In [ ]:
def rebuild_text(tokens, trailing_whitespace):
    text = ''
    for token, ws in zip(tokens, trailing_whitespace):
        ws = " " if ws == True else ""
        text += token + ws
    return text

"""
# rebuild edilen text'in çıktısı
for index, row in df.iterrows():
    text = rebuild_text(row['tokens'], row['trailing_whitespace'])
    print(text)
"""

"""
---------------------------------
- rebuild_text fonksiyonu metindeki tokenleri, metindeki sırası ve aralarında boşluk olcak 
  şekilde text değişkenine atıyor ve bunu return ediyor.
---------------------------------
"""

In [ ]:
def split_rows(df, max_length, doc_stride):
    new_df = []
    for _, row in df.iterrows():
        tokens = row['tokens']
        if len(tokens) > max_length:
            start = 0
            while start < len(tokens):
                remaining_tokens = len(tokens) - start
                if remaining_tokens < max_length and start != 0:
                    # Adjust start for the last window to ensure it has max_length tokens
                    start = max(0, len(tokens) - max_length)
                end = min(start + max_length, len(tokens))
                new_row = {}
                new_row['document'] = row['document']
                new_row['tokens'] = tokens[start:end]
                new_row['trailing_whitespace'] = row['trailing_whitespace'][start:end]
                new_row['token_indices'] = list(range(start, end))
                new_row['full_text'] = rebuild_text(new_row['tokens'], new_row['trailing_whitespace'])
                new_df.append(new_row)
                if remaining_tokens >= max_length:
                    start += doc_stride
                else:
                    # Break the loop if we've adjusted for the last window
                    break
        else:
            new_row = {
                'document': row['document'], 
                'tokens': row['tokens'], 
                'trailing_whitespace': row['trailing_whitespace'], 
                'token_indices': row['token_indices'], 
                'full_text': row['full_text']
            }
            new_df.append(new_row)
    return pd.DataFrame(new_df)

stride_df = split_rows(df, max_length, doc_stride)

# bölünmüş metinden oluşturulan df çıktısı:
# print(stride_df)

"""
---------------------------------
- "split_rows" belgeleri daha küçük parçalara bölmeyi sağlıyor ve "max_length" yani
  parçaların max uzunluğunu ve "doc_stride" belgelerin nasıl bölüneceğini parametre olarak alıyor

+ boş bir "new_df" oluşturulur, bu değişken belge parçalarını içericek.
+ döngü ile df'deki tüm belgeleri işlemeye başlar, tüm tokenler işlenene kadar devam eder
+ token sayısı maksimum uzunluktan büyükse belgeleri daha küçük parçalara böler
+ belgenin son kısmının maximum uzunluktan kısa olup olmadığını kontrol ediyor, eğer kısaysa son kısmı maximum uzunluğa 
    sahip olacak şekilde düzenler böylelikle tüm parçalar birbirleri ile aynı uzunluğa eşit olur  
+ ardından "new_row" değişkeni oluşturuyor ve orjinal belgedeki "document", "tokens", "trailling_whitespaces", "token_indices" 
    ve "full_text" özellikleri bu değişkene kopyalanıyor ardından "new_df" adlı df'e ekliyor
+ return değeri olarak oluşturulan bu yeni dataframe yani "new_df" return ediliyor
+ oluşturduğumuz "split_rows" fonksiyonu çağırılıyor ve sonucu "stride_df" değişkenine atanıyor
---------------------------------
"""

<div style="background-color:purple; color:white; padding:10px; border-radius: 5px; text-align:center;">
  <h1>Kod blok no: 2</h1>
</div>

In [ ]:
import numpy as np
from datasets import Dataset

def tokenize(example, tokenizer, label2id, max_length):

    # rebuild text from tokens
    text = []
    token_map = []
    labels = []
    
    idx = 0

    for t, ws in zip(
        example["tokens"], example["trailing_whitespace"]
    ):
        text.append(t)
        token_map.extend([idx]*len(t))

        if ws:
            text.append(" ")
            token_map.append(-1)
            
        idx += 1

    # actual tokenization
    tokenized = tokenizer("".join(text), return_offsets_mapping=True, max_length=max_length)
    text = "".join(text)
    length = len(tokenized.input_ids)

    return {**tokenized, "length": length, "token_map": token_map,}

"""
---------------------------------
- fonksiyon metni tokenize etmemizi sağlar ve tokenize edilmiş metnin uzunluğunu tutar, tokenize edilmiş metinle birlikte ek 
    bilgiler içeren bir sözlük döndürür.
  
+ "text", "token_map", "labels" adında boş listeler oluşturuluyor sırasıyla tokenlerin metin karşılıklarını, tokenlerin 
    belge içindeki konumlarını ve etiketleri saklar.
+ for döngüsü başlatılır ve bu döngüde tokenler ile ("tokens") arasındaki boşluklar ("trailing_whitespace") birlikte 
    döndürülür her döngüde, her bir token "text" listesine eklenir ve tokenin konumu da "token_map" listesine eklenir.
+ eğer tokenin boşluğu varsa boşluk karakteri de "text" listesine eklenir ve "token_map" listesine bu sebeple "-1" eklenir
+ #actual tokenization kısmında "text" listesindeki tüm elemanlar birleştirilerek metin dizisi oluşturuluyor ve bu dizi tokenizere
    iletilerek tokenize edilmiş çıktı alır ve "return_offsets_mapping=True" ayarı ile tokenlerin orjinal metindeki konumlarının da 
    döndürülmesini ister. tokenize edilmiş çıktı tokenized değişkenine atanır.
+ daha sonrasında "text" değişkenine tokenize edilmiş metin atanır, "length" değişkenine tokenize edilmiş metnin uzunluğu atanır
+ Return olarak çıktıya ek bilgiler eklenerek sözlük oluşturulmuş, tokenized, uzunluk ve "token_map" return edilir
---------------------------------
"""

def create_dataset(data, tokenizer, max_length, label2id):
    ds = Dataset.from_dict({
        "full_text": data.full_text.tolist(),
        "document": data.document.tolist(),
        "tokens": data.tokens.tolist(),
        "trailing_whitespace": data.trailing_whitespace.tolist(),
        "token_indices": data.token_indices.tolist(),
    })
    ds = ds.map(tokenize, fn_kwargs={"tokenizer": tokenizer, "label2id": label2id, "max_length": max_length}, num_proc=3)
    return ds

"""
---------------------------------
- bu fonksiyon sözlükten (dictionary) veri kümesi oluşturmamıza yarar

+ her bir key, bir öznitelik yani attiribute olarak kabul edilir
+ fonksiyonda oluşturulan ds sözlüğüne tokenize fonksiyonu ile tokenizasyon işlemi yapılır 
+ son olarak da oluşturulan tokenize edilmiş data return edilir
---------------------------------
"""

In [ ]:
id2label = {0: 'B-EMAIL', 1: 'B-ID_NUM', 2: 'B-NAME_STUDENT', 3: 'B-PHONE_NUM', 4: 'B-STREET_ADDRESS', 5: 'B-URL_PERSONAL', 6: 'B-USERNAME', 7: 'I-ID_NUM', 8: 'I-NAME_STUDENT', 9: 'I-PHONE_NUM', 10: 'I-STREET_ADDRESS', 11: 'I-URL_PERSONAL', 12: 'O'}
label2id = {v:k for k,v in id2label.items()}
"""
---------------------------------
- etiketleri id'lerine id'lerin de etiketlere karşılık geldiği iki sözlük oluşturulmuş

+ ilk satırda idlere karşılık gelen etiketlerin sözlüğü oluşturulmuş örneğin 0 idli etiket
    "B-EMAIL"e karşılık geliyor ve bunlar "id2label" değişkeninde tutuluyor.
+ ikinci satırda "id2label" değişkenindeki key ve values değerleri ters çevirilerek etiketlere
    karşılık gelen idlerin olduğu bir sözlük oluşturulmuş ve "label2id" değişkenine atanmış
---------------------------------
"""

"""
# sözlüklerin çıktısı: 
print("id2label=>",id2label)
print("-------------------------------------------")
print("label2id=>",label2id)
"""

In [ ]:
valid_ds = create_dataset(stride_df, tokenizer, inference_max_length, label2id)
"""
---------------------------------
- yukarıda oluşturduğumuz create_dataset fonksiyonu çağırılarak veri kümesi oluşturuluyor

+ "label2id" değişkeni de parametre olarak gönderiliyor yani etiketlerin idlerinden yararlanılarak sözlük oluşturuluyor
---------------------------------
"""

"""
# oluşturulan veri kümesinin df olarak ekrana basılmış hali:
import pandas as pd
valid_df = valid_ds.to_pandas()
print(valid_df)
"""

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model, 
    data_collator=collator, 
    tokenizer=tokenizer,
)
"""
---------------------------------
- transformers kütüphanesinden trainer kullanılarak modelin doğrulama veri kümesi üzerinde tahmin yapması sağlanmış.
  
+ trainer için atama yapılırken kullanılan "model", doğrulama işlemi için kullanılacak modeldir
+ "data_collator" veri kümesini modelin girişine uygun hale getiren kollektör
+ "tokenizer" ise metini tokenize etmek için kullanılır
---------------------------------
"""

In [ ]:
preds = trainer.predict(valid_ds)
"""
---------------------------------
- oluşturulan trainer nesnesi kullanılarak veri kümemiz üzerinde tahmin yapılması sağlanmış

+ oluşan tahminleri "preds" değişkenine atamış
---------------------------------
"""

<div style="background-color:purple; color:white; padding:10px; border-radius: 5px; text-align:center;">
  <h1>Kod blok no: 3</h1>
</div>

In [ ]:
# https://www.kaggle.com/code/valentinwerner/915-deberta3base-inference?scriptVersionId=161126788

import numpy as np
import pandas as pd
from IPython.core.display import HTML
import wandb
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm.auto import tqdm
import spacy
from spacy.tokens import Span
from spacy import displacy

def parse_predictions(predictions, id2label, ds, threshold=0.9):
    
    pred_softmax = np.exp(predictions) / np.sum(np.exp(predictions), axis = 2).reshape(predictions.shape[0],predictions.shape[1],1)
    preds = predictions.argmax(-1)
    preds_without_O = pred_softmax[:,:,:12].argmax(-1)
    O_preds = pred_softmax[:,:,12]
    preds_final = np.where(O_preds < threshold, preds_without_O , preds)
    """
    ---------------------------------
    - bu fonksiyon oluşturup "preds" değişkenine atadığımız tahminleri alır ve tahminlerden çıkan varlıkları işler
      modelin çıkardığı tahminler, modelimizdeki eşik değerinden düşük (threshold değişkeni yani 0.9) olan "O" etiketlerini 
      diğer en olası etiketler ile değiştiriyor.

    + Eğer bir tahminin olasılığı belirli bir eşik değerinden düşükse (threshold değişkeni yani 0.9) ve bu tahmin "O" etiketine sahipse, 
      bu tahmin diğer en olası etiketlerle değiştirilir. Bu şekilde, modelin anlamlı varlıkları tespit etme yeteneği artırılır ve modelin doğruluğu iyileştirilir.
    + fonksiyon sonrası elde edilen df modelin tahminlerini ve tahminlere ilişkin varlıkları içerir
    + "pred_softmax" değişkeni modelin tahmin ettiği etiket olasılıklarını softmax fonksiyonu ile normalize etmiş
    + "preds" değişkenine her bir tahmin için en yüksek olasılığa sahip etiketin indeksi belirlenmiş
    + "preds_without_0" değişkenine "o" etiketi dışındaki en yüksek olasılığa sahip etiketin indeksi belirlenmiş
    + "O_preds" değişkenine "O" etiketine ait olasılıklar belirlenmiş
    + "preds_final" değişkeninde "O" etiketine ait olasılıkların eşik değerimizden küçük olup olmadığına bakılmış
        eğer küçükse "O" etiketi yerine "O" olmayan en yüksek olasılığa sahip etiketin indeksi kullanılmış ve tahminler elde edilmiş
    --------------
    """
    triplets = []
    row, document, token, label, token_str = [], [], [], [], []
    for i, (p, token_map, offsets, tokens, doc, indices) in enumerate(zip(preds_final, ds["token_map"], ds["offset_mapping"], ds["tokens"], ds["document"], ds["token_indices"])):

        for token_pred, (start_idx, end_idx) in zip(p, offsets):
            label_pred = id2label[token_pred]

            if start_idx + end_idx == 0: continue

            if token_map[start_idx] == -1:
                start_idx += 1

            # ignore "\n\n"
            while start_idx < len(token_map) and tokens[token_map[start_idx]].isspace():
                start_idx += 1
            """
            --------------
            + "triplets" adında, sonuçların depolanması için boş liste oluşturulmuş
            + for döngüsü ve zip fonksiyonu ile son tahminler yani "preds_final" "token_map" "offset_mapping" "tokens" "document" 
                "token_indices" verilerinde döngü başlatılmış
            + iç içe döngü oluşturulmuş, iç döngü her bir tokeni ve offsetlerini alıp her bir token ve tahmin edilen etiketini işlemek için kullanılmış
            + tahmin edilen etiketin indeksi id2label kullanılarak "label_pred" değişkenine atanmış
            + eğer tokenin offsetlerinin toplamı 0 ise yani token belgede yer almıyorsa döngü atlanıp sonraki tokene geçilmiş
            + tokenin başlangıcında boşluk var ise boşluk karakterini atlamak için start_idx arttırılmış
            + while döngüsü ile boşluk karakterleri atlanarak, metin belgesindeki tokenin başlangıç offseti belirlenmiş
                başlangıç indeksi belirli bir boşluk karakteri değilse veya metin belgesinin sonunda değilse arttırılmış.
            --------------
            """
    
            if start_idx >= len(token_map): break

            original_token_id = token_map[start_idx]
            token_id = indices[original_token_id]

            # ignore "O" predictions and whitespace preds
            if label_pred != "O" and token_id != -1:
                triplet = (label_pred, token_id, tokens[original_token_id])

                if triplet not in triplets:
                    row.append(i)
                    document.append(doc)
                    token.append(token_id)
                    label.append(label_pred)
                    token_str.append(tokens[original_token_id])
                    triplets.append(triplet)
    """
    ----------------
    + koşul ile başlangıç indeksi metin belgesinin sonuna ulaştıysa döngü sonlandırılmış
    + "original_token_id" değişkenine tokenin indisi belirlenmiş
    + "token_id" değişkenine tokenin indeksi atanmış
    + if ile eğer tahmin edilen etiket "O" değilse ve tokenin indeksi -1 değilse yani boşluk karakteri değilse
        etiket, tokenin indeksi, orjinal metin belgesindeki token  "triplet" değişkenine atanır
    + "triplet" değişkenine atanan değer daha önce oluşturduğumuz "triplets" değişkeninde değilse listeye eklenir
    + sonuç olarak triplets listesi, metin belgesindeki her bir token için tahmin edilen etiketleri ve bu etiketlere 
        karşılık gelen tokenlerin indislerini içerir. Ancak, bu listeye sadece "O" olmayan etiketler ve boşluk karakterleri eklenir.
    ----------------
    """
    df = pd.DataFrame({
        "eval_row": row,
        "document": document,
        "token": token,
        "label": label,
        "token_str": token_str
    })

    df = df.drop_duplicates().reset_index(drop=True)

    df["row_id"] = list(range(len(df)))
    return df
"""
------------------------------
+ triplets lsitesindeki sonuçlar pandas df'ine dönüştürülmüş
+ dönüştürülürken tekrar eden yani duplikat satırlar kaldırılmış ve indeksler sıfırlanmış
+ "row_id" satırı oluşturulmuş. her bir satırın benzersiz kimliğini ifade ediyor
+ row_id eklenmiş bir şekilde df return edilir
---------------------------------
"""

In [ ]:
preds_df = parse_predictions(preds.predictions, id2label, valid_ds, threshold=threshold)
"""
---------------------------------
- "parse_predictions" fonksiyonu kullanılarak modelin tahminleri "preds_df" değişkenine işlenmiş 
---------------------------------
"""


# modelin tahminlerinin çıktısı:
# print(preds_df)

In [ ]:
preds_df[["row_id", "document", "token", "label"]].to_csv("submission.csv", index=False)
"""
---------------------------------
- "preds_df" değişkenine işlediğimiz verilerin sadece "row_id", "document", "token", "label" sütunları alınarak
  CSV dosyasına dönüştürülmüş
  
+ "to_csv" yöntemi ile bu df "submission.csv" dosyasına yazdırılmış
---------------------------------
"""